### Jewel Osco meat prices

In [13]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

header = { 
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:139.0) Gecko/20100101 Firefox/139.0'
}
page_url = 'https://www.jewelosco.com/aisle-vs/meat-seafood/meat-essentials.html?sort=price'

# Set up Selenium
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:139.0) Gecko/20100101 Firefox/139.0")

# Get page with Selenium
driver = webdriver.Chrome(options=options)
driver.get(page_url)

# exhaust load more button
last_count = 0
same_count = 0
max_attempts = 5

while same_count < 3 and max_attempts > 0:
    max_attempts -=1

    # load products and find total count of products
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a[data-qa="prd-itm-pttl"]')))
    current_products = driver.find_elements(By.CSS_SELECTOR, 'a[data-qa="prd-itm-pttl"]')
    current_count = len(current_products)

    # check product count to see if load more worked (loaded more items)
    if current_count == last_count:
        same_count += 1
    else:
        same_count = 0
        last_count = current_count

    # check button existence
    load_more = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.bloom-load-button')))
    driver.execute_script("arguments[0].click();", load_more)
    time.sleep(3)

# Parse with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'lxml')
products = soup.findAll('a', attrs={'data-qa':'prd-itm-pttl'})

if not products:
    print("No products found. Possible issues:")
    print("- The page structure may have changed")
    print("- The content might be loaded dynamically with JavaScript")
    print("- The website might be blocking your scraper")
else:
    for product in products:
        print(product.text.strip())

driver.quit()

TimeoutException: Message: 
